# **PDF Yükleme**

In [43]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("FutbolKuralları.pdf");
data = loader.load()

In [44]:
print(f"Toplam sayfa sayısı: {len(data)}")

Toplam sayfa sayısı: 230


In [ ]:
# print("\n10. Sayfanın içeriği:\n")
# print(data[38].page_content[:5000])  

# **PDF Metnini Tek Bir Bütün Haline Getirme**

In [46]:
# PDF'nin tüm sayfalarını tek bir metin haline getir
all_text = "\n".join([page.page_content for page in data])

# **Veriyi parçalara ayırma (Chunking İşlemi)**
### Metni 1000 karakterlik parçalar halinde bölerek işlemeye hazır hale getiriyoruz.

In [48]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Metni daha küçük parçalar (chunk'lar) haline getirecek olan text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Parçalara bölelim
docs = text_splitter.split_text(all_text)

# Kaç parça olduğunu görelim
print(f"Toplam parça sayısı: {len(docs)}")

Toplam parça sayısı: 216


In [ ]:
# docs[4]

# **Metinleri Vektörleştirme (Embedding) ve Veritabanına Kaydetme**
### Metinleri embedding (vektör) formatına çevirip ChromaDB'ye kaydediyoruz.

In [50]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

# .env dosyasını yükle 
load_dotenv()

# Google Gemini Embedding modelini başlat
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# ChromaDB'yi başlat ve tüm chunk'ları vektörleştirerek kaydet
vectorstore = Chroma.from_texts(texts=docs, embedding=embeddings, persist_directory="./chroma_db")

print("Tüm chunk'lar embedding'e çevrildi ve ChromaDB'ye kaydedildi")

Tüm chunk'lar embedding'e çevrildi ve ChromaDB'ye kaydedildi


In [ ]:
# # Örnek bir sorguyu vektörleştir ve sonucu göster
# vector = embeddings.embed_query("hello, world!")
# print(vector[:5])  # İlk 5 elemanı yazdır

# **Retriever (Bilgi Getirici) Oluşturma**
###  Retriever ile, kullanıcının sorduğu soruya en uygun metinleri veritabanından getiriyoruz.

In [53]:
# ChromaDB'den verileri tekrar yükle
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

# Retriever oluştur (benzerlik araması yapacak)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# **LLM (Büyük Dil Modeli) ile RAG Zinciri Oluşturma**
### Retriever ile getirilen bilgileri kullanarak, Gemini API üzerinden cevap oluşturuyoruz.

In [71]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

# Google Gemini LLM'yi başlat
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.3,
    max_tokens=500
)

# sistem prompt'u
system_prompt = (
    "Sen bir yardımcı asistansın ve yalnızca futbol kuralları hakkında sorulara cevap veriyorsun. "
    "Yanıtlarını yalnızca verilen bağlam içeriğinden oluştur. "
    "Eğer sorunun cevabını bilmiyorsan, 'Bu konuda yardımcı olamıyorum.' de. "
    "Cevaplarını en fazla üç cümle ile ver ve doğru bilgi içerdiğinden emin ol.\n\n"
    "{context}"
)

# Prompt şablonunu oluştur
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# Question-Answer zincirini oluştur
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Retriever + LLM kombinasyonu ile RAG zincirini oluştur
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Deneme Sorusu

In [72]:
# Kullanıcıdan gelen soruyu işleyelim
response = rag_chain.invoke({"input": "Kırmızı kart nedir?"})

# Chatbot'un cevabını yazdır
print(response["answer"])

Kırmızı kart, bir oyuncunun ihracını belirtir.  Bir oyuncuya, yedek oyuncuya, değiştirilmiş oyuncuya veya takım görevlisine kırmızı kart gösterilebilir. İhraç gerektiren ihlaller arasında, rakip takımın oyunu tekrar başlatmasını geciktirmek veya teknik alanı izinsiz terk etmek gibi davranışlar yer alır.
